# 🥋 Lekcja 44: FSDP (Jak trenować giganty?)

W DDP pamięć jest ograniczona przez najsłabszą kartę.
W FSDP pamięć to **suma VRAM wszystkich kart**.

**Koncepcja ZeRO (Zero Redundancy Optimizer):**
Standardowy trening (Adam) zużywa pamięć na:
1.  **Parametry (Wagi):** fp32 (4 bajty).
2.  **Gradienty:** fp32 (4 bajty).
3.  **Stan Optymalizatora (Momentum + Variance):** fp32 (8 bajtów).

Razem: **16 bajtów na jeden parametr**.
Model 1B parametrów wymaga **16 GB VRAM** (tylko na "statykę", bez aktywacji!).

**Rozwiązanie FSDP:**
Podzielmy te 16GB na 8 kart graficznych. Każda trzyma tylko 2GB.
Kiedy potrzebujemy wag do obliczeń, robimy **All-Gather** (pobieramy resztę), a po obliczeniach natychmiast je kasujemy.

In [1]:
import torch
import torch.nn as nn

# Symulacja wielkiego modelu (Transformer)
# 100 milionów parametrów to mało dla LLM, ale dużo dla laptopa
class BigTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        # 12 warstw, model dimension 1024
        self.layers = nn.Sequential(*[
            nn.Linear(1024, 4096) for _ in range(25) # Dużo dużych warstw
        ])

    def forward(self, x):
        return self.layers(x)

model = BigTransformer()

# Liczymy parametry
total_params = sum(p.numel() for p in model.parameters())
print(f"Liczba parametrów: {total_params:,}")

Liczba parametrów: 104,960,000


## Kalkulator Pamięci VRAM

Zanim kupisz karty graficzne, musisz umieć policzyć, czy model się zmieści.
Napiszmy funkcję inżynierską, która to szacuje.

In [2]:
def estimate_memory(params_count, num_gpus=1, use_fsdp=False):
    # 1. Wagi (FP32) - 4 bajty
    weights_mem = params_count * 4
    
    # 2. Gradienty (FP32) - 4 bajty
    grads_mem = params_count * 4
    
    # 3. Optimizer (Adam trzyma 2 stany: momentum i variance) - 8 bajtów
    opt_mem = params_count * 8
    
    total_mem = weights_mem + grads_mem + opt_mem
    
    if use_fsdp:
        # FSDP dzieli to wszystko przez liczbę GPU!
        # (Teoretycznie idealne skalowanie)
        total_mem /= num_gpus
        
    # Konwersja na GB
    return total_mem / (1024**3)

print("--- SZACUNEK PAMIĘCI (Dla modelu 100M) ---")
print(f"1 GPU (DDP):    {estimate_memory(total_params, 1):.2f} GB VRAM")
print(f"4 GPU (DDP):    {estimate_memory(total_params, 4, use_fsdp=False):.2f} GB VRAM (Brak zysku pamięci!)")
print(f"4 GPU (FSDP):   {estimate_memory(total_params, 4, use_fsdp=True):.2f} GB VRAM (Zysk!)")

# A co z modelem GPT-3 (175 miliardów parametrów)?
gpt3_params = 175_000_000_000
print(f"\n--- GPT-3 (175B) ---")
print(f"Wymagane VRAM (1 GPU): {estimate_memory(gpt3_params, 1):.2f} GB")
print("Żadna karta tyle nie ma (A100 ma 80GB).")
print(f"Wymagane na kartę przy 64 GPU (FSDP): {estimate_memory(gpt3_params, 64, True):.2f} GB (To się zmieści!)")

--- SZACUNEK PAMIĘCI (Dla modelu 100M) ---
1 GPU (DDP):    1.56 GB VRAM
4 GPU (DDP):    1.56 GB VRAM (Brak zysku pamięci!)
4 GPU (FSDP):   0.39 GB VRAM (Zysk!)

--- GPT-3 (175B) ---
Wymagane VRAM (1 GPU): 2607.70 GB
Żadna karta tyle nie ma (A100 ma 80GB).
Wymagane na kartę przy 64 GPU (FSDP): 40.75 GB (To się zmieści!)


## Składnia FSDP (Wrapper)

W PyTorch FSDP działa podobnie do DDP – owijamy model klasą.
Ale jest haczyk: **`auto_wrap_policy`**.

Nie chcemy shardingować byle jak (np. przeciąć pojedynczy neuron na pół).
Chcemy shardingować całe bloki Transformera.
Policy mówi: *"Jeśli warstwa ma więcej niż 10mln parametrów, potnij ją i rozdziel na GPU"*.

In [3]:
# To jest kod poglądowy (wymaga środowiska rozproszonego do uruchomienia)
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
from torch.distributed.fsdp.wrap import size_based_auto_wrap_policy

def fsdp_wrapper_example(model):
    # Polityka: Owijaj (tnij) warstwy większe niż 10 milionów parametrów
    my_policy = lambda module, recurse, **kwargs: size_based_auto_wrap_policy(
        module, recurse, min_num_params=10_000_000, **kwargs
    )
    
    # Owijanie (na CPU przed wysłaniem na GPU, żeby oszczędzić pamięć przy starcie!)
    sharded_model = FSDP(
        model,
        auto_wrap_policy=my_policy,
        cpu_offload=None # Można ustawić na True, żeby zrzucić wagi do RAMu zwykłego!
    )
    
    return sharded_model

print("Kod FSDP gotowy (do użycia w skrypcie torchrun).")

Kod FSDP gotowy (do użycia w skrypcie torchrun).


## CPU Offloading (Ostatnia deska ratunku)

Co jeśli FSDP na 8 kartach to wciąż za mało?
FSDP ma asa w rękawie: **CPU Offload**.

Wagi leżą w tanim RAM-ie komputera (CPU).
Są przesyłane na GPU tylko w momencie, gdy są potrzebne do obliczeń (Forward/Backward), a potem natychmiast wracają do RAM.
*   **Zaleta:** Możesz trenować gigantyczne modele na słabych kartach.
*   **Wada:** Jest to wolne (wąskim gardłem jest szyna PCIe).

In [4]:
from torch.distributed.fsdp import CPUOffload

# Włączenie tej flagi pozwala trenować modele większe niż VRAM
offload = CPUOffload(offload_params=True)

print("CPU Offload skonfigurowany: Wagi będą żyły w RAMie, odwiedzając GPU tylko na chwilę.")

CPU Offload skonfigurowany: Wagi będą żyły w RAMie, odwiedzając GPU tylko na chwilę.


## 🥋 Black Belt Summary

1.  **DDP vs FSDP:**
    *   **DDP:** Szybkie, ale każdy GPU musi pomieścić cały model. (Dobre do ResNet50).
    *   **FSDP:** Wolniejsze (dużo komunikacji sieciowej), ale pozwala trenować modele większe niż pamięć GPU. (Konieczne do LLM).
2.  **ZeRO Stages (Odpowiedniki w DeepSpeed):**
    *   Stage 1: Sharding Stanu Optymalizatora (Największy zysk, mały narzut).
    *   Stage 2: Sharding Gradientów.
    *   Stage 3: Sharding Parametrów (Pełne FSDP).
3.  **Koszty:** FSDP wymaga szybkiej sieci między kartami (NVLink), inaczej karty będą czekać na przesyłanie kawałków modelu.